In [32]:
import pandas as pd
import string
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import gensim
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import string
import ast

In [31]:
df = pd.read_csv("final_data.csv")
df

,title,author,description,genres
0,The Hunger Games,Suzanne Collins,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas..."
1,Harry Potter and the Order of the Phoenix,"J.K. Rowling, Mary GrandPr (Illustrator)",There is a door at the end of a silent corrido...,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',..."
2,To Kill a Mockingbird,Harper Lee,The unforgettable novel of a childhood in a sl...,"['Classics', 'Fiction', 'Historical Fiction', ..."
3,Pride and Prejudice,"Jane Austen, Anna Quindlen (Introduction)",Alternate cover edition of ISBN 9780679783268S...,"['Classics', 'Fiction', 'Romance', 'Historical..."
4,Twilight,Stephenie Meyer,About three things I was absolutely positive.F...,"['Young Adult', 'Fantasy', 'Romance', 'Vampire..."
...,...,...,...,...
46972,Fractured,Cheri Schmidt (Goodreads Author),The Fateful Trilogy continues with Fractured. ...,"['Vampires', 'Paranormal', 'Young Adult', 'Rom..."
46973,Anasazi,Emma Michaels,"'Anasazi', sequel to 'The Thirteenth Chime' by...","['Mystery', 'Young Adult']"
46974,Marked,Kim Richardson (Goodreads Author),--READERS FAVORITE AWARDS WINNER 2011--Sixteen...,"['Fantasy', 'Young Adult', 'Paranormal', 'Ange..."
46975,Wayward Son,"Tom Pollack (Goodreads Author), John Loftus (G...",A POWERFUL TREMOR UNEARTHS AN ANCIENT SECRETBu...,"['Fiction', 'Mystery', 'Historical Fiction', '..."


### Gộp và xử lý các cột title, author, description 

In [20]:
df['text'] = df['title'] + ' ' + df['author'] + ' ' + df['description']
lemmatizer = WordNetLemmatizer()

# Hàm tiền xử lý văn bản
def preprocess_text(text):
    # Chuyển thành chữ thường
    text = text.lower()
    # Loại bỏ ký tự đặc biệt
    text = ''.join([char for char in text if char not in string.punctuation])
    # Tokenization và loại bỏ stopwords
    words = text.split()
    words = [word for word in words if word not in ENGLISH_STOP_WORDS]
    # Lemmatization
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)

# Áp dụng tiền xử lý cho các cột 'title', 'author', 'description'
df['text']= df['text'].astype(str)
df['text'] = df['text'].apply(preprocess_text)


In [21]:
df['text']

0        hunger game suzanne collins winning mean fame ...
1        harry potter order phoenix jk rowling mary gra...
2        kill mockingbird harper lee unforgettable nove...
3        pride prejudice jane austen anna quindlen intr...
4        twilight stephenie meyer thing absolutely posi...
                               ...                        
46972    fractured cheri schmidt goodreads author fatef...
46973    anasazi emma michael anasazi sequel thirteenth...
46974    marked kim richardson goodreads author reader ...
46975    wayward son tom pollack goodreads author john ...
46976    daughter helaman misty moncur goodreads author...
Name: text, Length: 46977, dtype: object

### Embedding cột text

In [23]:
model = Word2Vec(sentences=df['text'], vector_size=200, window=5, min_count=1, sg=0)

# Get word embeddings for each description
def get_embedding(text_tokens):
    # Get embeddings for each word in the description and average them
    embeddings = [model.wv[word] for word in text_tokens if word in model.wv]
    if embeddings:
        return sum(embeddings) / len(embeddings)
    else:
        return [0] * model.vector_size  # Return a zero vector if no words are in the model

# Apply to create embedding column
df['embedding'] = df['text'].apply(get_embedding)

### Tiền xử lý nhãn sách

In [33]:
df['genres'] = df['genres'].apply(ast.literal_eval)


In [34]:
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['genres'])

In [39]:
y[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [37]:
import csv
y.to_csv('check.csv',index = False)

AttributeError: 'numpy.ndarray' object has no attribute 'to_csv'